# Predicción de la Radiación Solar

Vamos a intentar predecir la radiación solar dependiendo distintos factores como la hora del día, la temperatura, presión, etc..

Pueden encontrar este dataset en Kaggle: 

https://www.kaggle.com/dronio/SolarEnergy

Tener una herramienta que prediga la cantidad de radiación solar que habrá, dependiendo diversos factores, nos permitirá ver si es rentable o no invertir en paneles solares para alimentar nuestros aparatos electros hogareños, que cantidad de energía vamos a tener disponible durante el año y distintas ventajas relacionadas con el tema que un especialista en el área podria descubrir.

In [ ]:
#Importamos las librerias
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
#Cargamos los datos
df_SolarRad = pd.read_csv("../input/SolarEnergy/SolarPrediction.csv")

In [ ]:
df_SolarRad.head()

In [ ]:
df_SolarRad.shape

# Preprocesamiento

Vamos a trabajar sobre la columna **Time**, vamos a tomar los horarios y transformarlos en nuevos features (**Morning, Afternoon, Night** y **EarlyMorning**)

In [ ]:
#Cortamos los datos de la columna tomando los primeros 2 valores y transformandolos en enteros
df_SolarRad["Time"] = df_SolarRad.Time.str.slice(stop=2).astype(int)

In [ ]:
df_SolarRad["Time"].describe()

Vemos que los datos se tomaron en distintos horarios.

Vamos a realizar la transformación a nuevos features 

In [ ]:
#Creamos la nueva columna y la llenamos con valores Booleanos en caso de que cumpla la condicion
df_SolarRad["Morning"] = (df_SolarRad["Time"] >= 6) & (df_SolarRad["Time"] <= 12)
df_SolarRad["Afternoon"] = (df_SolarRad["Time"] >= 13) & (df_SolarRad["Time"] <= 19)
df_SolarRad["Night"] = (df_SolarRad["Time"] >= 20) & (df_SolarRad["Time"] <= 23)
df_SolarRad["EarlyMorning"] = (df_SolarRad["Time"] >= 0) & (df_SolarRad["Time"] <= 5)

In [ ]:
#Vemos que se crearon correctamente
df_SolarRad.head(2)

Para que esto funcione tenemos que transformar los valores Booleanos a númericos, para esto vamos a utilizar una herramienta muy útil de la librería sklearn: LabelEncoder

Esta herramienta nos permite transformar y etiquetar categorías a variables numéricas. Generalmente se utiliza cuando tenemos un problema de Clasificación sobre nuestra variable objetivo (**y**), pero en este caso nos va a ser útil.

Es recomendable aprender mas sobre esta técnica: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [ ]:
from sklearn.preprocessing import LabelEncoder

lab_enc = LabelEncoder()
#Utilizamos fit_transform ya que entrenamos y aplicamos el cambio sobre el mismo conjunto de datos
df_SolarRad['Morning'] = lab_enc.fit_transform(df_SolarRad['Morning'])
df_SolarRad['Afternoon'] = lab_enc.fit_transform(df_SolarRad['Afternoon'])
df_SolarRad['Night'] = lab_enc.fit_transform(df_SolarRad['Night'])
df_SolarRad['EarlyMorning'] = lab_enc.fit_transform(df_SolarRad['EarlyMorning'])

In [ ]:
df_SolarRad[["Time","Morning","Afternoon","Night","EarlyMorning"]].tail()

Vemos que el cambio se aplico correctamente, a la hora 0 tenemos el valor 1 que nos indica que es de madrugada 

Vamos a hacer lo mismo con las columnas **TimeSunRise** y **TimeSunSet**

In [ ]:
df_SolarRad["TimeSunRise"] = df_SolarRad.TimeSunRise.str.slice(stop=2).astype(int)
df_SolarRad["TimeSunSet"] = df_SolarRad.TimeSunSet.str.slice(stop=2).astype(int)

In [ ]:
df_SolarRad["TimeSunRise"].describe()

In [ ]:
df_SolarRad["TimeSunSet"].describe()

Vemos que los datos de estas dos columnas no varían, asique no van a sumar valor aplicarlos al modelo

In [ ]:
#Eliminamos las columnas
df_SolarRad = df_SolarRad.drop(columns=["TimeSunSet","TimeSunRise"])

Veamos si tenemos **outliers**

In [ ]:
plt.figure(1, figsize=(10,6)) 
plt.title("Radiation") 
sns.boxplot(df_SolarRad["Radiation"]) 

plt.figure(2, figsize=(10,6))
plt.title("Temperatura")
sns.boxplot(df_SolarRad["Temperature"])

plt.figure(3, figsize=(10,6)) 
plt.title("Presion")
sns.boxplot(df_SolarRad["Pressure"])

Vemos que son pocos. Podemos quitarlos a medida que entrenamos los modelos y probar como afecta al **score** de nuestro modelo

Vamos a visualizar la correlación de los datos con nuestro objetivo **Radiation**, esto nos va a permitir seleccionar que features son mas relevante para utilizarlas con nuestro modelo

Vamos a generar un mapa de calor utilizando Seaborn: https://seaborn.pydata.org/generated/seaborn.heatmap.html

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df_SolarRad.corr(),cmap='coolwarm',annot=True)

Vamos a filtrar el dataset con las columnas que tengan correlación alta o positiva

In [ ]:
X = df_SolarRad[["Temperature","Pressure","Morning","Afternoon"]] #Nuestros features mas relevantes
y = df_SolarRad["Radiation"] #Separamos nuestro objetivo

# Machine Learning

Vamos a probar distintos modelos de **regresión** a ver como funcionan nuestros modelos. Inicialmente vamos a cargar nuestros modelos con los **hiperparametros** por defecto, después vamos a pasar a la etapa de optimización.

In [ ]:
#Importamos librerias para evaluar nuestros modelos
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Primero vamos a dividir nuestro dataset en 4 partes:
 - X_train: Datos para entrenar el modelos 
 - y_train: Nuestro objetivo a predecir que utilizaremos para entrenar el modelo
 - X_test: Conjunto de datos al cual le aplicaremos la predicción
 - y_test: Conjunto de datos objetivo con las que compararemos los resultados de la predicción para ver que tan bien funciona el modelo
 
**y_pred**: Es el resultado del modelo aplicado al conjunto de X_test que se va a comparar con **y_test** para medir el error del modelo

Generalmente la divición se hace un 80%(train), 20%(test):

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

### Linear Regression

Ver hiperparametros del modelo: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
from sklearn.linear_model import LinearRegression

regression_linear = LinearRegression()

regression_linear.fit(X_train, y_train) #Entrenamos el modelo

In [ ]:
y_pred = regression_linear.predict(X_test) #Aplicamos la prediccion

Grafiquemos el valor de nuestra **predicción** (y_pred) y los **valores reales** (relación 1:1)

In [ ]:
plt.scatter(y_test,y_pred) #Valores predichos 
plt.plot(y_test, y_test, 'r') #Nuestro valor real
plt.xlabel('Valor Real', fontsize = 15)  
plt.ylabel('Prediccion', fontsize = 15)  
plt.show()

La linea roja son nuestros valores reales y los puntos azules nuestras predicciones.
Vemos que los puntos siguen el patrón de la linea roja, pero una gran cantidad cae fuera. 

Vamos a medir el error en nuestras predicciones con el **RMSE**

In [ ]:
#Aplicamos la raiz cuadrada (np.sqrt) al MSE para obtener el RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred)) 
print("RMSE: ", rmse)

Vemos que nuestro **error mínimo** es de **186.3**, esto quiere decir que nuestra predicción esta alejada 186 watts por metro^2 (unidad de medida de la radiación solar), del **valor real**.

Veamos como nos dio el R^2 de nuestro modelo

In [ ]:
print("R^2: ", r2_score(y_test, y_pred))

Es un valor bastante bajo, hay que tener en cuenta que aplicamos el modelo por defecto sin tunear los hiperparametros o realizar algún tipo de optimización.

Vamos a probar otros modelos de regresión a ver si esto mejora

### Decision Tree Regressor

Ver hiperparametros del modelo: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

In [ ]:
from sklearn.tree import DecisionTreeRegressor 

tree_reg = DecisionTreeRegressor(random_state =12)

tree_reg.fit(X_train, y_train) 

In [ ]:
y_pred = tree_reg.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse)

In [ ]:
plt.scatter(y_test,y_pred)
plt.plot(y_test, y_test, 'r')
plt.xlabel('Valor Real', fontsize = 15)  
plt.ylabel('Prediccion', fontsize = 15)  
plt.show()

In [ ]:
print("R^2: ", r2_score(y_test, y_pred))

Vemos que con el modelo de **Decision Tree** bajo nuestro **RMSE** a **158**, y nuestro **score** subió a 75, esto es una buena señal de mejora.

Sigamos viendo ejemplos con otros modelos

### Random Forest

Ver hiperparametros del modelo: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

En este caso ya definimos el hiperparamentro **n_estimators**, el cual es la cantidad de árboles a entrenar dentro de nuestro bosque

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regression_RF = RandomForestRegressor(n_estimators = 200, random_state =12)
regression_RF.fit(X_train, y_train)

In [ ]:
y_pred = regression_RF.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse)

In [ ]:
plt.scatter(y_test,y_pred)
plt.plot(y_test, y_test, 'r')
plt.xlabel('Valor Real', fontsize = 15)  
plt.ylabel('Prediccion', fontsize = 15)  
plt.show()

In [ ]:
r2_score(y_test, y_pred)

Vemos que el RMSE y el R2 no cambiaron

# Optimizacion

Existen varias técnicas para optimizar nuestros modelos
- Cross Validation: https://scikit-learn.org/stable/modules/cross_validation.html
- Randomized Search: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html
- Gradient Descent: Es mas utilizado en **Deep Learning**
- Grid Search: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Vamos a utilizar este ultimo

### GridSearch

Esta técnica se basa en **comparar los hiperparametros** del modelo, cargados en una grilla que nosotros definimos, Grid Search nos va a brindar los mejores parámetros a utilizar

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#Cargamos la grilla con los hiperparametros a comparar
param_grid ={'max_depth': [4, 6, 8, 10, 12], 'max_features': [1, 2, 3, 4]}

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=12)
#Pasamos los parametros a GridSearch 
grid_search = GridSearchCV(tree_reg, param_grid, cv=5,
                           scoring='r2', 
                           return_train_score=True)

In [ ]:
#Entrenamos
grid_search.fit(X_train, y_train)

In [ ]:
#Vemos los resultados de GridSearch 
results = pd.DataFrame(grid_search.cv_results_)
results.head()

In [ ]:
print("El mejor score es:", grid_search.best_score_) 
print("Mejores parametros entcontrados:\n", grid_search.best_estimator_)

Vemos que para este conjunto de datos, el mejor score es **75** con la métrica de **R2**

Vamos a utilizar el mejor estimador que nos brindo GridSearch

In [ ]:
optimised_Tree = grid_search.best_estimator_

In [ ]:
#Tomamos un ejemplo del conjunto de test
test_predict= X_test[50:51]
test_predict

In [ ]:
print("Radiacion solar:",optimised_Tree.predict(test_predict), "watts por metro^2")